<a href="https://colab.research.google.com/github/genetooo/ia/blob/main/mainzinho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# passo a passo


# passo 0: entender a empresa e o seu desafio
# passo 1: impotar a base de dados
import pandas as pd
import numpy as np
import json


tabela = pd.read_csv('treino.csv')
display(tabela)


,id,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,target
0,12251,38.763463,4.372530,2.888516,12.352539,18.906550,9.605924,3.456596,3.959477,4.738227,3.692509,11.192263,9.076242,4.692962,2
1,4529,32.192181,3.158109,2.134617,16.193530,26.786878,9.347958,5.093277,6.893790,3.399840,2.080853,19.486692,12.349952,3.217670,0
2,11284,28.370782,3.006067,1.969280,15.163950,28.515109,7.946870,5.609813,6.511194,3.256799,2.290993,18.405205,14.781329,3.231217,0
3,10993,38.215409,2.174232,2.935104,11.501867,18.419571,7.954005,4.229832,4.522060,4.869234,3.383514,12.782517,17.343742,5.009182,2
4,12927,38.904545,2.956164,2.253149,12.282552,25.547887,8.168776,3.653026,6.809110,4.971130,2.127363,19.247327,20.115183,3.560524,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10495,6951,30.605226,2.172545,1.456766,15.916774,33.872292,7.766799,4.455944,4.866775,5.274495,3.194181,13.756923,15.133962,4.489715,4
10496,8097,37.153719,2.930280,2.514520,10.042402,22.815845,6.018057,5.359753,5.656202,3.601738,2.760592,15.988397,19.794919,3.752870,2
10497,9011,24.316856,2.771631,2.018644,16.082389,27.999121,6.014674,6.209202,4.304756,3.905507,3.502867,12.168262,6.530749,5.191197,0
10498,12617,44.748792,3.174414,1.447737,13.578811,33.966677,8.671664,4.754359,7.941178,3.325981,1.505581,22.447345,21.038993,1.795912,0


In [8]:
# passo 2: preparar a base de dados para a IA
tabela.info()

# y e quem eu quero prever
y = tabela['target']

# x e o que eu tenho para prever
x = tabela.drop(columns=['id','target'])

#separar dados de treino e dados de teste
from sklearn.model_selection import train_test_split
x_treino, x_teste, y_treino, y_teste = train_test_split(x,y)

# comferindo a calidade dos dados
from sklearn.preprocessing import StandardScaler

# Criar o normalizador (usando StandardScaler, pois funciona bem para SVM e redes neurais)
scaler = StandardScaler()

# Normalizar apenas para os modelos que precisam
x_treino_normalizado = scaler.fit_transform(x_treino)
x_teste_normalizado = scaler.transform(x_teste)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10500 entries, 0 to 10499
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      10500 non-null  int64  
 1   col_0   10500 non-null  float64
 2   col_1   10500 non-null  float64
 3   col_2   10500 non-null  float64
 4   col_3   10500 non-null  float64
 5   col_4   10500 non-null  float64
 6   col_5   10500 non-null  float64
 7   col_6   10500 non-null  float64
 8   col_7   10500 non-null  float64
 9   col_8   10500 non-null  float64
 10  col_9   10500 non-null  float64
 11  col_10  10500 non-null  float64
 12  col_11  10500 non-null  float64
 13  col_12  10500 non-null  float64
 14  target  10500 non-null  int64  
dtypes: float64(13), int64(2)
memory usage: 1.2 MB


In [9]:
# passo 3: treinar a IA
# criar modelo

#distancia
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

#importar metricas
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix



In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# --- Ajuste refinado para KNN ---
param_grid_knn = {
    'n_neighbors': [4, 5, 6, 7, 8],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

grid_knn = GridSearchCV(
    estimator=KNeighborsClassifier(),
    param_grid=param_grid_knn,
    cv=5,
    scoring='f1_macro',
    n_jobs=-1
)
grid_knn.fit(x_treino_normalizado, y_treino)
best_knn_params = grid_knn.best_params_
print("Melhores parâmetros KNN:", best_knn_params)

# Treinamento final com os melhores parâmetros para KNN
modelo_knn = KNeighborsClassifier(**best_knn_params)
modelo_knn.fit(x_treino_normalizado, y_treino)




Melhores parâmetros KNN: {'metric': 'euclidean', 'n_neighbors': 8, 'weights': 'distance'}


KNeighborsClassifier(metric='euclidean', n_neighbors=8, weights='distance')

In [11]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

# Grade de parâmetros reduzida
param_grid_svm = {
    'C': np.logspace(-1, 2, 4), # Menos valores para C
    'kernel': ['rbf'], # Kernel linear se apropriado
    'gamma': ['scale', 0.1, 1], # Menos valores para gamma
}

# RandomizedSearchCV para explorar o espaço de parâmetros de forma mais eficiente
grid_svm = RandomizedSearchCV(
    estimator=SVC(class_weight='balanced', probability=True, random_state=42),
    param_distributions=param_grid_svm, # Use param_distributions em vez de param_grid
    n_iter=10, # Número de iterações
    cv=3, # Número reduzido de folds de validação cruzada
    scoring='f1_macro',
    n_jobs=-1,
    random_state=42
)

grid_svm.fit(x_treino_normalizado, y_treino)
best_svm_params = grid_svm.best_params_
print("Melhores parâmetros SVM:", best_svm_params)

# Treinamento final com os melhores parâmetros para SVM
modelo_svm = SVC(**best_svm_params, class_weight='balanced', probability=True, random_state=42)
modelo_svm.fit(x_treino_normalizado, y_treino)

Melhores parâmetros SVM: {'kernel': 'rbf', 'gamma': 0.1, 'C': np.float64(10.0)}


SVC(C=np.float64(10.0), class_weight='balanced', gamma=0.1, probability=True,
    random_state=42)

In [12]:
# Avaliação dos modelos
previsao_knn = modelo_knn.predict(x_teste_normalizado)
previsao_svm = modelo_svm.predict(x_teste_normalizado)

print(classification_report(y_teste, previsao_knn))
print(classification_report(y_teste, previsao_svm))

              precision    recall  f1-score   support

           0       0.83      0.93      0.88      1262
           1       0.75      0.69      0.72       413
           2       0.67      0.69      0.68       350
           3       0.67      0.40      0.50       196
           4       0.76      0.64      0.69       404

    accuracy                           0.78      2625
   macro avg       0.73      0.67      0.69      2625
weighted avg       0.77      0.78      0.77      2625

              precision    recall  f1-score   support

           0       0.91      0.83      0.87      1262
           1       0.70      0.76      0.73       413
           2       0.66      0.75      0.70       350
           3       0.53      0.63      0.58       196
           4       0.72      0.69      0.70       404

    accuracy                           0.77      2625
   macro avg       0.70      0.73      0.72      2625
weighted avg       0.78      0.77      0.78      2625



In [13]:
# Criar um dicionário com os melhores hiperparâmetros
melhores_hiperparametros = {
    'KNN': best_knn_params,
    'SVM': best_svm_params
}

import json

# Salvar os hiperparâmetros em um arquivo JSON
with open('melhores_hiperparametros1.json', 'w', encoding='utf-8') as arquivo_json:
    json.dump(melhores_hiperparametros, arquivo_json, indent=4, ensure_ascii=False)

print("Hiperparâmetros salvos em formato JSON com sucesso!")

Hiperparâmetros salvos em formato JSON com sucesso!


In [14]:
# qual o melhor modelo de IA com ajustes

previsao_knn = modelo_knn.predict(x_teste_normalizado)
previsao_svm = modelo_svm.predict(x_teste_normalizado)

print(classification_report(y_teste, previsao_knn))
print(classification_report(y_teste, previsao_svm))

print(confusion_matrix(y_teste, previsao_knn))
print(confusion_matrix(y_teste, previsao_svm))

              precision    recall  f1-score   support

           0       0.83      0.93      0.88      1262
           1       0.75      0.69      0.72       413
           2       0.67      0.69      0.68       350
           3       0.67      0.40      0.50       196
           4       0.76      0.64      0.69       404

    accuracy                           0.78      2625
   macro avg       0.73      0.67      0.69      2625
weighted avg       0.77      0.78      0.77      2625

              precision    recall  f1-score   support

           0       0.91      0.83      0.87      1262
           1       0.70      0.76      0.73       413
           2       0.66      0.75      0.70       350
           3       0.53      0.63      0.58       196
           4       0.72      0.69      0.70       404

    accuracy                           0.77      2625
   macro avg       0.70      0.73      0.72      2625
weighted avg       0.78      0.77      0.78      2625

[[1178   25   17   12 

In [15]:
# passo 5: usar o melhor modelo para faser a previsao
# qual o melhor modelo?

#importar tabela de novos cliemtes
tabela_para_teste = pd.read_csv('teste.csv')

# Armazena a coluna 'id' separadamente
ids = tabela_para_teste["id"]

# Remove a coluna 'id' antes da previsão
tabela_para_teste_sem_id = tabela_para_teste.drop(columns=['id'], errors='ignore')

# Faz a previsão
nova_previsao_knn = modelo_knn.predict(tabela_para_teste_sem_id)
nova_previsao_svm = modelo_svm.predict(tabela_para_teste_sem_id)

# Criar um DataFrame com os IDs e as previsões
resultado_knn = pd.DataFrame({"id": ids, "target": nova_previsao_knn})
resultado_svm = pd.DataFrame({"id": ids, "target": nova_previsao_svm})
display(tabela_para_teste)

display(nova_previsao_knn)
display(nova_previsao_svm)

# Exibir o resultado
display(resultado_knn)
display(resultado_svm)

# Salvando para CSV
df_knn = pd.DataFrame(resultado_knn)

df_knn.to_csv("resposta.knn.csv", index=False, header=True, encoding="utf-8")

df_svm = pd.DataFrame(resultado_svm)

df_svm.to_csv("resposta.svm.csv", index=False, header=True, encoding="utf-8")


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


,id,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12
0,2142,37.555079,3.205281,1.583984,11.330132,32.542511,6.581602,5.524091,5.202632,3.288405,3.009713,14.706292,20.530879,2.889453
1,3846,28.933095,3.905861,2.171001,14.644065,26.406560,9.515010,4.023027,6.528586,4.202300,2.281440,18.454367,14.862851,3.183574
2,6177,37.082903,3.259937,1.887675,12.313557,29.368109,6.533770,5.088085,5.913968,3.925079,2.619016,16.717026,16.871735,3.512345
3,11779,24.011327,4.599539,2.102587,17.349264,27.121683,12.231295,2.664319,5.226405,5.232794,2.996656,14.773491,9.133299,3.739929
4,1593,31.403592,2.543982,0.533288,14.778941,43.525198,5.225097,7.100292,4.516120,2.753415,3.386776,12.765727,16.675742,3.000528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,11496,37.789367,3.511433,2.814992,10.697124,19.675079,9.844242,3.994824,5.082404,3.762631,3.075747,14.366443,20.097503,3.278142
4496,8856,24.889203,2.508254,2.525974,15.868246,22.696114,6.931519,7.380456,6.599213,1.865822,2.242648,18.654010,10.229343,3.687254
4497,4937,39.208404,2.843656,1.975679,14.071951,28.448221,7.391533,5.525009,3.912253,3.788423,3.718447,11.058774,10.266351,4.531029
4498,7574,24.559490,2.084494,3.359952,12.531981,13.978740,6.302277,6.162282,4.496384,3.523565,3.397616,12.709938,12.636072,5.461903


array([0, 0, 0, ..., 0, 1, 0])

array([0, 0, 0, ..., 0, 0, 0])

,id,target
0,2142,0
1,3846,0
2,6177,0
3,11779,0
4,1593,0
...,...,...
4495,11496,0
4496,8856,0
4497,4937,0
4498,7574,1


,id,target
0,2142,0
1,3846,0
2,6177,0
3,11779,0
4,1593,0
...,...,...
4495,11496,0
4496,8856,0
4497,4937,0
4498,7574,0


In [16]:
import pickle

# Salvando o modelo treinado
with open('modelo_knn_treinado.pkl', 'wb') as f:
    pickle.dump(modelo_knn, f)

print("Modelo salvo com sucesso!")
with open('modelo_svm_treinado.pkl', 'wb') as f:
    pickle.dump(modelo_svm, f)

print("Modelo salvo com sucesso!")
# Carregando o modelo salvo
with open('modelo_knn_treinado.pkl', 'rb') as f:
    modelo_knn_carregado = pickle.load(f)
with open('modelo_svm_treinado.pkl', 'rb') as f:
    modelo_svm_carregado = pickle.load(f)
# Agora você pode usar o modelo carregado para fazer previsões
y_pred_knn = modelo_knn_carregado.predict(x_teste_normalizado)
y_pred_svm = modelo_svm_carregado.predict(x_teste_normalizado)


Modelo salvo com sucesso!
Modelo salvo com sucesso!
